In [1]:
import numpy as np
import pandas as pd

import warnings 
warnings.filterwarnings('ignore')

# keyword extraction
import re
from keybert import KeyBERT
kw_model = KeyBERT(model='all-mpnet-base-v2')

#zero shot classification
from transformers import pipeline

In [2]:
data = pd.read_excel("data.xlsx")

In [3]:
#KeyBert
def keybert_pipeline_mono(text):
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(1, 1),
                                     #keyphrase_ngram_range=(2, 2),
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)

    return keywords


def keybert_pipeline_mono_no_scores(text):
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(1, 1),
                                     #keyphrase_ngram_range=(2, 2),
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)
    keywords_list= list(dict(keywords).keys())

    return keywords_list

In [4]:
data['KeyBert_mono'] = data["excerpt"].apply(keybert_pipeline_mono)
data['KeyBert_mono_no_scores'] = data["excerpt"].apply(keybert_pipeline_mono_no_scores)
data['KM'] = data['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[0])
data.to_excel('data with keywords.xlsx')

In [5]:
#zero shot classification
classifier_pipeline = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")

In [6]:
# Robotics

def zeroshotPipeline(text):
    input_ids = text
    #candidate_label = ['Ontology', 'Data Science', 'Astrophysics', 'Marine Biology', 'Climate Change', 'Psychology', 'War', 'Artificial Intelligence', 'Machine Learning', 'Scrum Master', 'Contemporary Art', 'Vegan Cooking', 'Landscape Photography', 'Switzerland', 'European Union', 'NATO', 'Family Constellations', 'Bioengineering', 'Medicine', 'Physics', 'Mathematics', 'Data Visualization', 'Topic Modeling', 'Antarctica', 'Sequoia', 'Blue Whale', 'Matcha', 'Solar Panels', 'Thyroid', 'Gender equality', 'Education', 'Developing country', 'Spanish History', 'Dog training', 'Solar System', 'Autoimmune disease']
    candidate_label = ['lego',
                        'mindstorm',
                        'computer science',
                        'technology',
                        'cinema',
                        'motors',
                        'programming',
                        'geometry',
                        'engineering',
                        'others']

    results = classifier_pipeline(input_ids, 
                         candidate_label)
                        # multi_label = True)
                        # hypothesis_template = template)
    score_id = np.argmax(results["scores"])
    final_label = results["labels"][score_id]
    prob = results["scores"][score_id]
    return final_label, prob


Robotics = data.loc[data['topic'] == 'Robotics']
Robotics["zero_shot_class_1"] = ""
#Robotics['zero_shot_class_1'] = Robotics["Reviews"].apply(zeroshotPipeline)
Robotics['zero_shot_class_1'] = Robotics["excerpt"].apply(zeroshotPipeline)
split_robotics = pd.DataFrame(Robotics['zero_shot_class_1'].tolist(), columns=['level-1', 'level-1 score'])
Robotics.reset_index(drop=True, inplace=True)
split_robotics.reset_index(drop=True, inplace=True)
data1 = pd.concat([Robotics, split_robotics], axis=1)
robotics = data1.copy()
robotics.to_excel('robotics_new_class.xlsx')

In [10]:
robotics['kw_1'] = robotics['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[0])
robotics['kw_2'] = robotics['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[1])
robotics['kw_3'] = robotics['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[2])
robotics['kw_4'] = robotics['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[3])
robotics['kw_5'] = robotics['keybert_mono_cleaned'].apply(lambda x : re.findall(r"[\w']+",x)[4])

col_list = ['topic', 'excerpt', 'level-1', 'kw_1', 'kw_2', 'kw_3', 'kw_4', 'kw_5']
data_final = robotics[col_list]
robotics.to_csv('robotics_new_class.csv')